In [ ]:
c=0
import math
from functools import reduce
for i in range(2,2+1):
    if i<=6:
        if i<=2:
            l=[1]*(6-i)+[2]
            c+=factorial(len(l))//reduce(lambda x,y:factorial(l.count(x))*factorial(l.count(y)),set(l))
            t=2
            while i*t<=6:
                l=[i]*t+[1]*(6-i*t)
                nr=factorial(len(l))
                if len(set(l))==1:
                    dr=factorial(len(l))
                else:
                    dr=reduce(lambda x,y:factorial(l.count(x))*factorial(l.count(y)),set(l))
                k=nr//dr
                t+=1

In [29]:
from math import factorial
from functools import reduce
class kk():
    def bk(cls,input1,input2):
        p=1
        for i in range(2,input2+1):
            if i<input1:
                if i<=input2:
                    l=[1]*(input1-i)+[i]
                    p+=factorial(len(l))//reduce(lambda x,y:factorial(l.count(x))*factorial(l.count(y)),set(l))
                    t=2
                    while i*t<=input1:
                        l=[i]*t+[1]*(input1-i*t)
                        nr=factorial(len(l))
                        if len(set(l))==1:
                            dr=factorial(len(l))
                        else:
                            dr=reduce(lambda x,y:factorial(l.count(x))*factorial(l.count(y)),set(l))
                        k=nr//dr
                        p+=k
                        t+=1
        return p

                

In [30]:
ob=kk()
ob.bk(6,2)

13

In [19]:
l=[1,1,1,1,2]
from functools import reduce
from math import factorial
k=factorial(len(l))//reduce(lambda x,y:factorial(l.count(x))*factorial(l.count(y)),set(l))
print(k)
i=2
t=1
while i*t<=6:
    l=[i]*t+[1]*(6-i*t)
    
    nr=factorial(len(l))
    if len(set(l))==1:
        dr=factorial(len(l))
    else:
        dr=reduce(lambda x,y:factorial(l.count(x))*factorial(l.count(y)),set(l))
    print(l,nr,dr)
    k=nr//dr
    print(k)
    t+=1


5
[2, 1, 1, 1, 1] 120 24
5
[2, 2, 1, 1] 24 4
6
[2, 2, 2] 6 6
1


In [1]:
import redis

In [2]:
redis.__version__

'4.3.3'

In [4]:
import struct 

def byteToInt(data):
    if data is not None:
        data = int.from_bytes(data, "big")
    return data
def getBinaryVal(data, position):
    binaryString = bin(data).replace("0b", "")
    # Adjsuting the length of BinaryString to 8 bits
    binaryString = "0" * (8 - len(binaryString)) + binaryString
    return int(binaryString[position])


def hex2float(data):
    return struct.unpack('f', data)[0]

In [97]:
 struct.unpack('f', bytearray(b'#\xdb\xc7C'))[0]

399.7120056152344

In [2]:
from datetime import datetime

def parseme(data):
    parsedObject = {
        u'imei': data[17:32].decode(),
        u'counter': byteToInt(data[3:4]),
        u'protocolVersion': data[13:17].decode(),
    }
    parsedObject[u'dieselVol'] = byteToInt(data[52:56])
    parsedObject[u'DTE'] = byteToInt(data[64:66])

    DD = str(byteToInt(data[47:48]))
    MM = str(byteToInt(data[46:47]))
    YYYY = str(byteToInt(
        data[44:45]) * 100 + byteToInt(data[45:46]))
    hh = str(byteToInt(data[40:42]))
    mm = str(byteToInt(data[42:43]))
    ss = str(byteToInt(data[43:44]))
    dateString = DD + " " + MM + " " + YYYY + " " + hh + " " + mm + " " + ss
    date = datetime.strptime(dateString, "%d %m %Y %H %M %S")
    # --------------------NOTE For statusBitmap2--------------------------

    statusBitmap2 = byteToInt(data[69:70])
    parsedObject[u'isGpsValid'] = getBinaryVal(statusBitmap2, 0)
    # parsedObject[u'isNetwork'] = getBinaryVal(statusBitmap2, 1)
    # parsedObject[u'isPacketLive'] = getBinaryVal(statusBitmap2, 2)
    parsedObject[u'isIgnitionOn'] = getBinaryVal(statusBitmap2, 3)
    parsedObject[u'isRelayCommExec'] = getBinaryVal(statusBitmap2, 6)

    # --------------------NOTE For statusBitmap3--------------------------

    isInlet = getBinaryVal(byteToInt(data[70:71]), 6)
    print("Inlet = ",isInlet)
    isTheft = getBinaryVal(byteToInt(data[70:71]), 7)
    print("Theft = ",isTheft)
    if isTheft == 1 and isInlet == 0:
        parsedObject[u'fuelProfile'] = {
            u'category': "Theft",
            u'quantity': byteToInt(data[56:60]),
            u'time': datetime.timestamp(date),
        }
    elif isTheft == 0 and isInlet == 1:
        parsedObject[u'fuelProfile'] = {
            u'category': "Inlet",
            u'quantity': byteToInt(data[56:60]),
            u'time': datetime.timestamp(date),
        }
    else:
        parsedObject[u'fuelProfile'] = None
    statusBitmap4 = byteToInt(data[71:72])
    parsedObject[u'FromDashmaster'] = {
        u'isAccGyroValid': getBinaryVal(statusBitmap4, 0),
        u'isMEMSPresent': getBinaryVal(statusBitmap4, 1)
    }
    statusBitmap5 = byteToInt(data[72:73])
    parsedObject[u'FromDEUnit'] = {
        u'isAccGyroValid': getBinaryVal(statusBitmap5, 0),
        u'isMEMSPresent': getBinaryVal(statusBitmap5, 1)
    }
    # -------------------NOTE statusBitmap6-----------------------------------
    # NOTE For Cement Mixer Status
    statusBitmap6 = byteToInt(data[73:74])
    if getBinaryVal(statusBitmap5, 0) == '1':
        parsedObject[u'rotateDirection'] = "Clkwise"
    else:
        parsedObject[u'rotateDirection'] = "Anticlkw"
    parsedObject[u'TimeStampPair'] = getBinaryVal(
        byteToInt(data[73:74]), 1)

    if parsedObject[u'isGpsValid'] == 1:
        rawLat = data[75:87].decode()
        rawLng = data[87:99].decode()
        speed = data[99:103]
        parsedObject[u'speed'] = hex2float(speed)
        parsedObject[u'rawLat'] = rawLat
        parsedObject[u'rawLng'] = rawLng

    parsedObject[u'timestamp'] = date
    parsedObject[u'temperature'] = byteToInt(data[66:68])

    parsedObject[u'acc_x1'] = byteToInt(data[103:105])
    parsedObject[u'acc_y1'] = byteToInt(data[105:107])
    parsedObject[u'acc_z1'] = byteToInt(data[107:109])
    parsedObject[u'gyro_x1'] = byteToInt(data[109:111])
    parsedObject[u'gyro_y1'] = byteToInt(data[111:113])
    parsedObject[u'gyro_z1'] = byteToInt(data[113:115])
    parsedObject[u'acc_x2'] = byteToInt(data[115:117])
    parsedObject[u'acc_y2'] = byteToInt(data[117:119])
    parsedObject[u'acc_z2'] = byteToInt(data[119:121])
    parsedObject[u'gyro_x2'] = byteToInt(data[121:123])
    parsedObject[u'gyro_y2'] = byteToInt(data[123:125])
    parsedObject[u'gyro_z2'] = byteToInt(data[125:127])
    parsedObject[u'acc_x3'] = byteToInt(data[127:129])
    parsedObject[u'acc_y3'] = byteToInt(data[129:131])
    parsedObject[u'acc_z3'] = byteToInt(data[131:133])
    parsedObject[u'gyro_x3'] = byteToInt(data[133:135])
    parsedObject[u'gyro_y3'] = byteToInt(data[135:137])
    parsedObject[u'gyro_z3'] = byteToInt(data[137:139])

    parsedObject[u'cementMixture'] = {
        u'clkw': byteToInt(data[139:141]),
        u'anticlkw': byteToInt(data[141:143]),
        u'startrotate': byteToInt(data[143:147]),
        u'endrotate': byteToInt(data[147:151]),
        u'timetorotate': byteToInt(data[151:155])
    }

    parsedObject[u'batteryVolt'] = byteToInt(data[155:156])
    parsedObject[u'internalVolt'] = byteToInt(data[156:157])
    parsedObject[u'lastSongId'] = byteToInt(data[157:158])

    # if vehicleId != None:
    #     parsedObject[u'vehicleId'] = vehicleId
    # if driverId != None:
    #     parsedObject[u'driverId'] = driverId
    for key,value in parsedObject.items():
        print(f"{key}={value}")
    print(parsedObject)

In [3]:
data=bytearray(b'!\x00\xa0\x00\x01\x00\x0c:*\x14\x16\x07\x1eGP10888888888888881\x17\x07A\x00\x00\x00|\x01\x00\x0c:*\x14\x16\x07\x1e\x00\x00\x01\xf4?\xe5\xc7C\x00\x00\x00\x00\x00\x00\x00\x06\x01^\x00\x17|\xf6\x00\x86\n\x86\x0117.36854000078.429010000)\\?@\x00\x02\x00\x05\x00\x01\x00\x03\x00\n\x00\x07\x00\x04\x00\x00\x00\x01\x00\x03\x00\x00\x00\x08\x00\x02\x00\x01\x00\t\x00\x03\x00\x04\x00\n\x00 \x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c\x04\xde\xff\xff')

In [7]:
speed = data[99:103]
hex2float(speed),hex2float(b'\xaeGQ@')

(2.990000009536743, 3.2699999809265137)

In [94]:
data=bytearray(b'!\x00\xa0\x00\x01\x00\x12\x191\x14\x16\x07\x1aGP10888888888888881\xcd.A\x00\x00\x00|\x01\x00\x12\x191\x14\x16\x07\x1a\x00\x00\x01\xf4\x00\x00\x01\xa4\x00\x00\x00\x00\x00\x00\x00\x06\x01^\x00\x17|\xf6\x80\x86\n\x86\x0117.36854000078.429010000\x1f\x85;@\x00\x06\x00\x03\x00\x08\x00\x08\x00\x04\x00\x05\x00\x0c\x00\x06\x00\x0b\x00\x07\x00\x01\x00\x08\x00\x03\x00\x05\x00\x02\x00\t\x00\x08\x00\x07\x00 \x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c\x04\xde\xff\xff')
subframe=data[34:len(data)-2]


In [95]:
# oo=str(byteToInt(data[4:5]))
# hh = str(byteToInt(data[5:6]))
# mm = str(byteToInt(data[6:7]))
# ss = str(byteToInt(data[7:8]))
# DD = str(byteToInt(data[12:13]))
# MM = str(byteToInt(data[11:12]))
# YYYY=str(byteToInt(data[9:10]))+str(byteToInt(data[10:11]))

# hh+mm+ss,YYYY+MM+DD
# int.from_bytes(Data, "big")
data[17:32]

bytearray(b'888888888888881')

In [96]:
len(bytearray(b'!\x00\xa0\x00\x01\x00\x12\x191\x14\x16\x07\x1aGP10888888888888881\xcd.A\x00\x00\x00|\x01\x00\x12\x191\x14\x16\x07\x1a\x00\x00\x01\xf4\x00\x00\x01\xa4\x00\x00\x00\x00\x00\x00\x00\x06\x01^\x00\x17|\xf6\x80\x86\n\x86\x0117.36854000078.429010000\x1f\x85;@\x00\x06\x00\x03\x00\x08\x00\x08\x00\x04\x00\x05\x00\x0c\x00\x06\x00\x0b\x00\x07\x00\x01\x00\x08\x00\x03\x00\x05\x00\x02\x00\t\x00\x08\x00\x07\x00 \x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c\x04\xde\xff\xff'))

160

In [62]:
data[5:6]

bytearray(b'\x00')

In [51]:
len(bytearray(b'!\x00\xa0\x00\x01\x00\x12\n\t\x14\x16\x07\x1aGP1088888888888881\x1c\x18A\x00\x00\x00|\x01\x00\x12\n\t\x14\x16\x07\x1a\x00\x00\x01\xf4\x00\x00\x01\xa4\x00\x00\x00\x00\x00\x00\x00\x06\x01^\x00\x17|\xf6\x80\x86\n\x86\x0117.36854000078.429010000\x9a\x99)@\x00\n\x00\t\x00\x07\x00\n\x00\x05\x00\x07\x00\x01\x00\x05\x00\x0c\x00\x07\x00\x00\x00\x02\x00\x0c\x00\x03\x00\x0b\x00\x01\x00\x01\x00\t\x00 \x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c\x04\xde\xff\xff'))

159

In [29]:
DD = str(byteToInt(data[47:48]))
MM = str(byteToInt(data[46:47]))
YYYY = str(byteToInt(
data[44:45]) * 100 + byteToInt(data[45:46]))
hh = str(byteToInt(data[40:42]))
mm = str(byteToInt(data[42:43]))
ss = str(byteToInt(data[43:44]))

In [30]:
MM

'26'

In [5]:
parseme(data)

ValueError: time data '0 26 2207 4107 57 20' does not match format '%d %m %Y %H %M %S'

In [6]:
 data[17:32].decode()

'88888888888881\x02'

In [69]:
data[99:103]
hex2float(data[99:103])

0.4000000059604645

In [75]:
k=12.86475
a = bytearray(struct.pack(">f", k))  
struct.unpack('>f', a)

(12.864749908447266,)

In [6]:
# data[15:30]
parseme(data)

ValueError: time data '0 0 0 0 0 0' does not match format '%d %m %Y %H %M %S'

In [45]:
import struct

value = 2555.91664673

ba = bytearray(struct.pack("f", value))  
ba

bytearray(b'\xab\xbe\x1fE')

In [46]:
struct.unpack('f', ba)

(2555.916748046875,)

In [33]:
float.hex(23.4)

'0x1.7666666666666p+4'

In [1]:
deviceStream:888888888888804

In [5]:
data={
  "data": "%,337,GP12,888888888888804,130403,20230123,03192,03192,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2214,N,07838.1710,E,004.35,130403,20230123,0000,0000,0000,020,020,020,030,24,1,$ 02353,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2212,N,07838.1640,E,001.08,185012,20230122,0000,0000,0000,020,020,020,030,24,1,$29093"
}

In [6]:
packetList=[]
for packet in data['data'].split('$')[:-1]:
        if len(packet) and packet[6] == '*':
            packetList.append(header+packet+'$')

In [7]:
packetList

[]

In [9]:
data2=b"%,337,GP12,888888888888804,130909,20230123,03277,03277,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2203,N,07838.1633,E,001.19,130909,20230123,0000,0000,0000,020,020,020,030,24,1,$"

In [11]:
data2

b'%,337,GP12,888888888888804,130909,20230123,03277,03277,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2203,N,07838.1633,E,001.19,130909,20230123,0000,0000,0000,020,020,020,030,24,1,$'

In [12]:
data1="%,334,GP12,869867033162936,120432,20221227,21595,21595,*XD,0000,00,00000,0000,0000,00000,0000,0000,00#,0,1719.4003,N,8207.2537,E,001.81,120432,20221227,0000,0000,0000,020,020,020,030,24,1,$"

In [15]:
data18804=b'%,337,GP12,888888888888804,131946,20230123,03461,03461,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2201,N,07838.1593,E,001.82,131946,20230123,0000,0000,0000,020,020,020,030,24,1,$ 06367,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,0000.0000,0,00000.0000,0,000.00,000080,20000101,0000,0000,0000,020,020,020,030,24,1,$52139'

In [16]:
data18804

b'%,337,GP12,888888888888804,131946,20230123,03461,03461,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2201,N,07838.1593,E,001.82,131946,20230123,0000,0000,0000,020,020,020,030,24,1,$ 06367,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,0000.0000,0,00000.0000,0,000.00,000080,20000101,0000,0000,0000,020,020,020,030,24,1,$52139'

In [5]:
def parseme(data):
    header=data[0:49].decode()
    Data = data[49:].decode()
    print(f"header=={header},Data=={Data}")
    print("data========",data)
    packetList=[]
    for packet in Data.split('$')[:-1]:
        print("packte===",packet)
        if len(packet) and packet[6] == '*':
            packetList.append(header+packet+'$')
    return packetList

In [8]:
data=b'%,336,GP12,888888888888804,133112,20230127,00197,00197,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2114,N,07838.1575,E,000.58,133112,20230127,0000,0000,0000,020,020,020,030,24,1,$00000,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,0000.0000,0,00000.0000,0,000.00,132033,20230127,0000,0000,0000,020,020,020,030,24,1,$56813'
pkts=parseme(data)


header==%,336,GP12,888888888888804,133112,20230127,00197,,Data==00197,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2114,N,07838.1575,E,000.58,133112,20230127,0000,0000,0000,020,020,020,030,24,1,$00000,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,0000.0000,0,00000.0000,0,000.00,132033,20230127,0000,0000,0000,020,020,020,030,24,1,$56813
data======== b'%,336,GP12,888888888888804,133112,20230127,00197,00197,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2114,N,07838.1575,E,000.58,133112,20230127,0000,0000,0000,020,020,020,030,24,1,$00000,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,0000.0000,0,00000.0000,0,000.00,132033,20230127,0000,0000,0000,020,020,020,030,24,1,$56813'
packte=== 00197,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2114,N,07838.1575,E,000.58,133112,20230127,0000,0000,0000,020,020,020,030,24,1,
packte=== 00000,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,0000.0000,0,00000.0000,0,000.00,132033,20230127,0000,0000,0000,020,020,020,030

In [25]:
import datetime as dt 
def parseMessage(data):
    data=data.split(',')
    date = None        
    parsedObject = {
        u'imei': data[3],
        u'counter': data[7],
        u'protocolVersion': data[2],
    }
    fuelHeight=int(data[14])*(0.1)
    parsedObject[u'fuelHeight']=fuelHeight
    invalidFuelRead="*XD,0000,00,00000,0000,0000,00000,0000,0000"
    fuelSensorRead=",".join(data[8:17])
    if invalidFuelRead==fuelSensorRead:
        parsedObject[u'dieselVol']=-1
        parsedObject[u'fuelHeight']=-1

    date_t=data[25]
    YYYY =date_t[0:4]
    MM =  date_t[4:6]
    DD = date_t[6:8]
    time_t=data[24]
    hh = time_t[0:2]
    mm = time_t[2:4]
    ss = time_t[4:6]
    date=dt.datetime(int(YYYY),int(MM),int(DD),int(hh),int(mm),int(ss))
    parsedObject[u'timestamp'] = date

    parsedObject[u'isGpsValid'] = int(data[34])
    parsedObject[u'isIgnitionOn'] = int(data[18])
    if parsedObject[u'isGpsValid'] == 1 :
        lat,lng=data[19],data[21]
        rawLat = (lat[0:2]+"."+str(float(lat[2:])/60).split('.')[1])[0:12]
        rawLng = (lng[0:2]+"."+str(float(lng[2:])/60).split('.')[1])[0:12]
        rawLat=rawLat if (data[20]=="N" or "E") else -rawLat
        rawLng=rawLng if (data[22]=="N" or "E") else -rawLng
        speed = float(data[23])
        parsedObject[u'speed'] = speed
        parsedObject[u'rawLat'] = rawLat
        parsedObject[u'rawLng'] = rawLng
    parsedObject[u'temperature'] = float(data[32])
    parsedObject[u'acc_x1'] = float(data[29])
    parsedObject[u'acc_y1'] = float(data[30])
    parsedObject[u'acc_z1'] = float(data[31])
    parsedObject[u'gyro_x1'] = float(data[26])
    parsedObject[u'gyro_y1'] = float(data[27])
    parsedObject[u'gyro_z1'] = float(data[28])
    parsedObject[u'acc_x2'] = float(data[29])
    parsedObject[u'acc_y2'] = float(data[30])
    parsedObject[u'acc_z2'] = float(data[31])
    parsedObject[u'gyro_x2'] = float(data[26])
    parsedObject[u'gyro_y2'] = float(data[27])
    parsedObject[u'gyro_z2'] = float(data[28])
    parsedObject[u'acc_x3'] = float(data[29])
    parsedObject[u'acc_y3'] = float(data[30])
    parsedObject[u'acc_z3'] = float(data[31])
    parsedObject[u'gyro_x3'] = float(data[26])
    parsedObject[u'gyro_y3'] = float(data[27])
    parsedObject[u'gyro_z3'] = float(data[28])
    parsedObject[u'batteryVolt'] = float(data[33])
    return parsedObject

In [26]:
pkts[0]

'%,336,GP12,888888888888804,133112,20230127,00197,00197,*XD,0000,00,00000,0000,0000,00000,0000,0000,80#,1,1744.2114,N,07838.1575,E,000.58,133112,20230127,0000,0000,0000,020,020,020,030,24,1,$'

In [27]:
parseMessage(pkts[0])

{'imei': '888888888888804',
 'counter': '00197',
 'protocolVersion': 'GP12',
 'fuelHeight': -1,
 'dieselVol': -1,
 'timestamp': datetime.datetime(2023, 1, 27, 13, 31, 12),
 'isGpsValid': 1,
 'isIgnitionOn': 1,
 'speed': 0.58,
 'rawLat': '17.736856666',
 'rawLng': '07.969291666',
 'temperature': 30.0,
 'acc_x1': 20.0,
 'acc_y1': 20.0,
 'acc_z1': 20.0,
 'gyro_x1': 0.0,
 'gyro_y1': 0.0,
 'gyro_z1': 0.0,
 'acc_x2': 20.0,
 'acc_y2': 20.0,
 'acc_z2': 20.0,
 'gyro_x2': 0.0,
 'gyro_y2': 0.0,
 'gyro_z2': 0.0,
 'acc_x3': 20.0,
 'acc_y3': 20.0,
 'acc_z3': 20.0,
 'gyro_x3': 0.0,
 'gyro_y3': 0.0,
 'gyro_z3': 0.0,
 'batteryVolt': 24.0}